## BRONZE TO SILVER LAYER


In [ ]:
import numpy as np
import pandas as pd
from common_utilities import GlobalPaths

In [ ]:
# Initialize TradeHistory Paths
TradeHistoryGlobalPath = GlobalPaths(source_name="DATA", object_name="TradeHistory")
TradeHistory_GOLD_LAYER_PATH = TradeHistoryGlobalPath.createLayer(layer_name="GOLD")
TradeHistory_GOLD_FILE_PATH = TradeHistory_GOLD_LAYER_PATH.joinpath("TradeHistory_data.csv")

# Initialize Investment Paths
InvestmentGlobalPath = GlobalPaths(source_name="DATA", object_name="Investment")
Investment_GOLD_LAYER_PATH = InvestmentGlobalPath.createLayer(layer_name="GOLD")
Investment_GOLD_FILE_PATH = Investment_GOLD_LAYER_PATH.joinpath("Investment_data.csv")

### SILVER Layer - Process Investment History

In [ ]:
# Read the CSV file
df_TradeHistory_GOLD = pd.read_csv(TradeHistory_GOLD_FILE_PATH)

# Filter the DataFrame
df_TradeHistory_GOLD = df_TradeHistory_GOLD[df_TradeHistory_GOLD["segment"].isin(["MF", "EQ"])]

# Convert datetime to date string
df_TradeHistory_GOLD["date"] = pd.to_datetime(df_TradeHistory_GOLD["datetime"]).dt.date

# Create a new DataFrame with an updated date range
date_range = pd.date_range(start=df_TradeHistory_GOLD["date"].min(), end=pd.to_datetime("today"), freq="D")
df_Investment_GOLD = pd.DataFrame({"date": date_range.date})

for stock_name, group in df_TradeHistory_GOLD.groupby("stock_name"):
    df_Investment_GOLD = pd.merge(df_Investment_GOLD, group[["date", "avg_price"]], on="date", how="left").rename(
        columns={"avg_price": stock_name},
    )

# sort the dataframe by date
df_Investment_GOLD = df_Investment_GOLD.sort_values(by=["date"])

# Reindexing to fill the missing data with the last available data
df_Investment_GOLD = df_Investment_GOLD.ffill()
df_Investment_GOLD.replace(0.0, np.nan, inplace=True)

# Save the result as a csv file
df_Investment_GOLD.to_csv(Investment_GOLD_FILE_PATH, index=None)
df_Investment_GOLD.info()
print("GOLD Layer csv file for Investment successfully created at:")
print(Investment_GOLD_FILE_PATH.resolve())